# Assignment 10
Multi-Resolution Analysis


1. Freesound Links of Sound Analyzed


a) https://www.freesound.org/people/GrowingUp/sounds/254114/

2) Band Edges used - (0-1500), (1500,5000), (5000,22050)
   Window Sizes - 4096, 2048, 1024
   
   The sound is a folk like instrument ensemble with percussion and vocal chants. The first band captured the lower frequencies with a higher resolution and the larger 2nd and 3rd bands were able to capture the time resolution better with a smaller window size. This enabled us to hear both the melodies of the vocal ensemble and the time resolution of the percussive hits.
   
   
b) https://www.freesound.org/people/antoinevg/sounds/368000/

2) Band Edges used - (0-500), (500,2000), (2000,22050)
   Window Sizes - 2048, 1024, 512
    
   The synth sound is a realy arpeggiated sound that needs really good time resolution for the on/off sound but the timbre of the synth keeps changing. The window sizes are chosen smaller to obtain the stronger time resolution especially in the wide range of 2000-22050 Hz. Using the smaller first two bands enables us to better resolve the really low harmonics (0-500Hz) and also the reasonably higher harmonics upto 2kHz.
   
   
3) A Smaller window is needed for better time resolution but this gives less frequency resolution while a Larger window is needed for better frequency resolution but averaging over a larrger segment in time. The multiresolution analysis obtains a balance of a good time and frequency resolution. This means that the peaks of a harmonic sound will be better resolved and this will help obtain a better HPS and HPR model. Addind many more bands will make it slightly more computationally expensive, since all the frames are now analyzed multiple times based on number of bands. But this is not significantly changed for 3 bands. O(mn) where m is number of bands, n is number of frames.

4) We will have better f0 tracking because of better resolved peaks. Sinetracking will become harder implementation wise, because we will need to keep track of sine tracks across bands in measuring freqDistance by accounting for the correct frequency resolution of each bin across the bands. A smaller band will have each bin corresponding to a larger frequency and a larger band will have each bin corresponding to a smaller frequency.

5) Other methods of better handling the time and frequency resolution problem would be by using (DWT) a discrete wavelet transform or by using the sparse representations[1] by using a over complete dictionary. Another method could be using deep learning techniques to learn the accurate multi-band lengths across each frame.


[1] (Berrıos, Rafael Alvarez. RIO PIEDRAS CAMPUS FACULTY OF NATURAL SCIENCES DEPARTMENT OF MATHEMATICS. Diss. UNIVERSITY OF PUERTO RICO, 2014.) 

In [ ]:
# Here is the Definition of the MultiResolution Analysis method in the SineModel.py
## This wont execute by itself. This needs to be added as a method in the SineModel.py in /sms-tools/software/models/

def sineModelMultiRes(x, fs, w_k, N_k, t, B_k):
        """
        Analysis/synthesis of a sound using the sinusoidal multiresolution model,
        x: input array sound, w_k: an array of three analysis windows, N_k: an array of three sizes of complex spectrum, t: threshold in negative dB, B_k: an array of bandwidths 
        returns y: output array sound
        """


        winLengths = np.array([w.size for w in w_k])

        Ns = 512                                                # FFT size for synthesis
        H = Ns/4                                                # Hop size used for synthesis
        hNs = Ns/2                                              # half of synthesis FFT size

        maxLength = int(math.floor((np.max(winLengths) + 1)/2.0))
        pin = max(hNs,maxLength)
        pend = x.size - pin

        #fftbuffer = np.zeros(Ns)                                # initialize buffer for FFT
        yw = np.zeros(Ns)                                       # initialize output sound frame
        y = np.zeros(x.size)                                    # initialize output array
        sw = np.zeros(Ns)                                       # initialize synthesis window
        ow = triang(2*H)                                        # triangular window
        sw[hNs-H:hNs+H] = ow                                    # add triangular window
        bh = blackmanharris(Ns)                                 # blackmanharris window
        bh = bh / sum(bh)                                       # normalized blackmanharris window
        sw[hNs-H:hNs+H] = sw[hNs-H:hNs+H] / bh[hNs-H:hNs+H]     # normalized synthesis window

        for w in w_k:
                w = w/sum(w)

        while pin < pend:

                ipfreqTotal = np.array([])
                ipmagTotal = np.array([])
                ipphaseTotal = np.array([])
                for w, N, B in zip(w_k, N_k, B_k):
                        fftbuffer = np.zeros(N)
                        M = w.size
                        hM1 = int(math.floor((M+1)/2))
                        hM2 = int(math.floor(M/2))

                        x1 = x[pin - hM1 : pin + hM2]
                        mX,pX = DFT.dftAnal(x1,w,N)
                        ploc = UF.peakDetection(mX, t)
                        iploc, ipmag, ipphase = UF.peakInterp(mX,pX,ploc)
                        ipfreq = fs*iploc/float(N)

                        band = np.where((ipfreq >= B[0]) & (ipfreq < B[1]))

                        ipfreq = np.take(ipfreq,band)[0]
                        ipmag = np.take(ipmag,band)[0]
                        ipphase = np.take(ipphase,band)[0]

                        ipfreqTotal = np.append(ipfreqTotal,ipfreq)
                        ipmagTotal = np.append(ipmagTotal,ipmag)
                        ipphaseTotal = np.append(ipphaseTotal,ipphase)

                Y = UF.genSpecSines(ipfreqTotal,ipmagTotal,ipphaseTotal,Ns,fs)
                fftbuffer = np.real(ifft(Y))
                yw[:hNs-1] = fftbuffer[hNs+1:]                        # undo zero-phase window
                yw[hNs-1:] = fftbuffer[:hNs+1]
                y[pin-hNs:pin+hNs] += sw*yw                           # overlap-add and apply a synthesis window
                pin += H                                              # advance sound pointer

        return y

